In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('2021-01-08.csv',parse_dates=['Date'],index_col='Date')
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [3]:
trading_days=250                  #Trading days per year (automated)

In [4]:
returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

In [5]:
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.004670,0.010079,0.022950,0.003701,-0.019795,0.009395,-0.015999,-0.003027,0.053465,-0.020043,...,-0.020953,0.004588,-0.001619,-0.006128,-0.006777,0.057034,-0.002407,-0.019236,-0.009251,0.000861
2,0.018784,0.000197,0.020427,0.006319,0.014751,0.010515,0.033349,0.010874,-0.017222,0.009435,...,0.009872,0.032987,0.011799,0.029049,0.014480,-0.013351,0.056647,0.004052,0.035903,0.057652


In [6]:
returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
    for i in range(0,len(df)):
      if returnso[cols][i] > 0:
        returnso[cols][i]=0               #Here, returnso would mean return considered for sortino ratio

In [7]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,-0.00467,0.0,0.0,0.0,-0.019795,0.0,-0.015999,-0.003027,0.000000,-0.020043,...,-0.020953,0.0,-0.001619,-0.006128,-0.006777,0.000000,-0.002407,-0.019236,-0.009251,0.0
2,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,-0.017222,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,-0.013351,0.000000,0.000000,0.000000,0.0


In [8]:
covmatsh=returnsh.cov()*trading_days      #Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso=returnso.cov()*trading_days      #Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio

In [9]:
num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [10]:
#2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
#Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

In [11]:
portfolio_returns = []                    #initializing an empty list for portfolio returns
portfolio_volatility =[]                  #initializing an empty list for portfolio risk
stock_weights =[]                         #initializing an empty list for portfolio weights
semi_deviation =[]                        #initializing an empty list for portfolio semi-deviation
sharpe =[]                                #initializing an empty list for portfolio sharpe ratio
sortino =[]                               #initializing an empty list for portfolio sortino ratio

In [12]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  return (a-c)/b                          #a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)

In [13]:
for single_portfolio in range(num_portfolios):                  #iterating forloop for 50000 times to generate 50000 portfolios
  weights = np.random.random(num_assets)                        #initializing random weights
  weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
  returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
  varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
  varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
  volatility_temp = np.sqrt(varsh)                              #portfolio risk
  semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
  shtemp = ratio(returns_temp,volatility_temp,risk_free_rate)   #calculating sharpe ratio
  sotemp = ratio(returns_temp,semi_temp,risk_free_rate)         #calculating sortino ratio
  portfolio_returns.append(returns_temp)                       
  portfolio_volatility.append(volatility_temp)
  stock_weights.append(weights)
  sharpe.append(shtemp)
  sortino.append(sotemp)
  semi_deviation.append(semi_temp)

In [14]:
portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Semi-Deviation' : semi_deviation, 'Sharpe Ratio' : sharpe, 
             'Sortino Ratio' : sortino}    
#here, 'portfolio' is a dictionary which will be used to create dataframe where each row will be a portfolio

In [15]:
for counter,symbol in enumerate(df.columns):
  portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights] 
#to the dictionary (named 'portfolio'), weights for each symbol are added in so as to be displayed in the dataframe

In [16]:
pc = pd.DataFrame(portfolio)         #making the final dataframe where data of 50000 portfolios is appended (subject to be saved, whose code is to be written)

In [17]:
pc=pc*100                                       #Converting everything to percentage
pc['Sharpe Ratio']=pc['Sharpe Ratio']/100       #leaving ratios as it is
pc['Sortino Ratio']=pc['Sortino Ratio']/100

In [18]:
#pc.to_csv('portfolios_by_MV.csv')  #saving the portfolios data

In [19]:
max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
max_sharpe_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
36608,181.958733,9.967081,3.308438,17.896788,53.916292,5.830609,8.587168,3.305578,6.0376,0.287612,...,6.550501,6.32158,0.321008,6.662619,6.999341,9.161689,2.725517,1.508763,1.592158,0.152508


In [20]:
max_sortino=pc['Sortino Ratio'].max()                                            #Best optimised portfolio wrt sortino ratio
max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
max_sortino_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
7455,226.315055,15.286879,2.478782,14.570342,89.85666,9.015889,0.544998,9.081724,7.232069,0.023027,...,3.402702,2.578976,6.768112,0.141978,3.181927,0.89806,0.529273,0.170531,6.682934,11.770929


In [21]:
#code for visualization is to be written

In [22]:
pc_sharpe=pc.drop(columns=['Sortino Ratio','Semi-Deviation'])

In [23]:
pc_sharpe_top10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).head(10)

In [24]:
pc_sharpe_top10.to_csv('Sharpe_Top10_MV.csv')

In [25]:
pc_sharpe_bottom10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).tail(10)

In [26]:
pc_sharpe_bottom10.to_csv('Sharpe_Bottom10_MV.csv')

In [27]:
pc_sharpe_bottom10


,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
34362,89.049369,18.087808,4.725247,0.494360,3.102726,0.303472,4.320193,8.687288,2.894446,9.324734,...,6.474212,1.587156,2.014765,7.180098,1.035217,3.725784,6.645178,9.243063,2.380741,1.146950
48309,79.742450,16.592584,4.590150,6.819209,0.546267,2.296652,1.756742,5.650866,6.732785,6.342526,...,5.600465,1.128915,3.121752,2.971990,6.376104,0.180278,1.773268,6.702088,3.963246,1.481385
46070,79.348556,16.790847,4.512492,9.005543,6.897689,1.495242,7.108736,7.191653,0.328693,3.785457,...,1.186840,2.010963,7.426924,6.119584,7.517305,1.080723,1.290675,3.926311,0.863448,2.144152
18389,83.369785,17.693882,4.509456,5.497131,0.772793,1.058609,4.187195,6.916688,3.241200,6.016361,...,8.474405,1.114450,5.424558,0.019835,4.835515,2.778165,0.714162,6.040497,5.716636,4.449858
24696,81.625930,17.397305,4.486093,5.114396,9.871042,0.697590,6.013359,7.380367,0.599949,6.152703,...,3.749522,2.325519,0.460905,9.886053,10.467062,1.150753,0.646679,5.986470,4.933236,0.030814
38029,101.296482,21.810472,4.480255,7.757431,0.330486,2.166252,4.014472,8.177993,1.234487,7.535021,...,8.194274,4.335250,3.382182,0.085526,0.697366,0.048944,5.412684,4.556617,7.712531,4.549596
15689,95.210295,20.467323,4.476907,6.935380,7.253389,0.839790,2.089156,5.040292,0.399576,7.192266,...,7.487994,3.182689,0.493710,3.738933,7.365983,1.111973,1.709733,3.933784,9.902782,1.580338
45822,77.532147,18.154958,4.073386,9.029674,0.823211,1.382259,3.639516,3.402557,1.963243,6.870039,...,9.365914,3.303262,1.363581,8.231841,8.570805,0.002298,2.607312,7.640750,1.095483,1.837660
31313,71.500542,16.737852,4.057901,7.742250,1.014548,0.072420,6.478392,3.924452,7.952237,2.366683,...,5.925598,0.827911,7.587622,2.341137,7.391614,0.455043,3.357966,10.405721,7.445282,1.060810
25277,58.630572,14.251189,3.862876,3.604254,5.889530,0.731506,6.177342,3.333757,9.905012,6.438556,...,8.576446,1.046964,1.202763,0.277483,3.836444,1.263206,0.722488,10.108781,2.299703,4.022140


In [28]:
pc_sharpe_top10

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
36608,181.958733,9.967081,17.896788,5.830609,8.587168,3.305578,6.037600,0.287612,7.876068,0.793630,...,6.550501,6.321580,0.321008,6.662619,6.999341,9.161689,2.725517,1.508763,1.592158,0.152508
26940,181.413879,9.939346,17.891909,2.379625,2.289313,6.533592,7.950619,3.109462,6.259577,1.257677,...,5.994676,4.107177,1.572585,1.186271,0.613680,9.882007,2.750768,0.773277,0.020065,3.660219
38904,170.791429,9.356934,17.870322,3.627646,4.953012,7.022986,6.415446,0.528737,8.350981,0.643901,...,0.679776,7.014254,3.588941,0.695168,3.117124,6.031224,1.215383,4.190818,4.364466,0.889783
18732,163.094234,8.933226,17.856285,0.988024,7.556399,7.091136,7.691688,10.186748,1.613291,8.840005,...,2.330079,0.793319,2.047082,6.707931,2.375130,8.401840,1.541501,3.440632,0.467278,0.772979
13827,172.951952,9.485352,17.856158,7.725252,5.296732,6.196898,0.861095,4.515975,0.534173,0.236829,...,0.302338,1.589136,6.278992,0.927074,1.025560,10.481424,3.518401,5.254110,3.663321,4.713995
11782,183.533546,10.094496,17.826897,2.051101,3.343436,8.737638,3.118035,4.707548,10.518381,1.609860,...,2.779648,0.463025,1.682043,0.818901,0.655038,8.378875,2.803171,2.279531,0.021394,6.442979
7366,158.788536,8.706936,17.825850,2.164559,9.657051,9.365114,7.270155,7.253378,6.848036,7.097455,...,2.045969,0.759316,0.779712,0.792440,10.588809,10.759045,1.541148,0.561949,0.110312,0.197628
27060,165.516308,9.092765,17.809358,0.494600,8.587441,8.379258,2.539655,7.282658,1.681213,5.650865,...,1.887147,0.693158,3.258497,1.572532,1.522133,8.817685,0.231439,3.527716,3.603933,3.014113
14232,142.487528,7.804653,17.798040,2.310081,7.000223,8.448618,7.237256,2.792958,2.277317,0.800346,...,4.174564,3.400201,2.525331,0.883396,5.618502,9.829516,4.298070,1.066482,0.007069,1.391426
49117,202.363514,11.185003,17.772325,1.120208,8.366417,9.893555,3.094181,0.036613,2.057233,0.549409,...,0.628642,3.376117,7.835038,0.611035,2.813696,9.802487,4.831734,1.235062,9.953675,3.483081


In [29]:
sharpe_optimal_portfolio=pc_sharpe_top10.head(1)

In [30]:
sharpe_optimal_portfolio.to_csv('Sharpe_Optimal_MV.csv')

In [31]:
sharpe_optimal_portfolio.T

,36608
Returns,181.958733
Standard Deviation,9.967081
Sharpe Ratio,17.896788
ASIANPAINT Weight,5.830609
BAJFINANCE Weight,8.587168
BAJAJFINSV Weight,3.305578
BRITANNIA Weight,6.037600
DIVISLAB Weight,0.287612
GRASIM Weight,7.876068
HCLTECH Weight,0.793630


In [32]:
pc_sortino=pc.drop(columns=['Sharpe Ratio','Standard Deviation'])

In [33]:
pc_sortino_top10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).head(10)

In [34]:
pc_sortino_top10.to_csv('Sortino_Top10_MV.csv')

In [35]:
pc_sortino_top10

,Returns,Semi-Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
7455,226.315055,2.478782,89.856660,9.015889,0.544998,9.081724,7.232069,0.023027,8.179124,3.668750,...,3.402702,2.578976,6.768112,0.141978,3.181927,0.898060,0.529273,0.170531,6.682934,11.770929
40407,210.282420,2.349840,87.964483,10.540729,5.944361,4.112972,7.147084,0.260947,0.680232,1.864073,...,0.640356,9.266575,6.327519,1.884717,4.638538,4.675897,6.466656,0.578346,4.258526,3.192103
12362,213.766548,2.489979,84.412978,2.044215,1.529866,0.735665,4.072079,0.177028,9.876725,0.012177,...,4.572853,6.141116,3.516285,5.092240,2.579222,1.026665,9.099380,0.053596,1.652184,10.188022
19579,215.569292,2.585606,81.988234,5.342051,10.776988,8.551642,1.698182,2.484724,7.276870,5.077976,...,2.104907,10.098426,0.606985,1.720601,1.840211,3.422702,5.197855,0.261417,0.421056,10.661851
24635,221.699565,2.660407,81.987306,6.372877,3.159168,6.640092,6.894560,0.684699,1.912730,5.183469,...,0.452013,6.702837,4.993955,3.774696,4.207206,7.615090,6.431870,3.295013,2.000461,6.472379
12885,201.070512,2.410225,81.938610,2.023792,9.251229,4.450206,9.441173,0.100418,10.920817,1.441390,...,0.311643,2.194366,3.465769,5.295316,2.676607,5.986114,3.235633,0.187491,9.480920,6.232539
19217,238.624977,2.937514,80.014931,3.859543,1.603211,8.361369,3.860303,0.134520,0.678274,4.717975,...,2.101338,7.633758,5.337538,7.679799,5.575415,6.380754,7.224411,0.453692,3.146000,7.771218
23267,217.024711,2.672808,79.857845,1.488301,9.054938,7.254731,1.803744,0.863035,1.682953,0.727481,...,0.807530,0.110404,1.906254,2.143808,2.850162,7.165560,8.507979,2.489907,6.285043,9.837192
35554,209.219458,2.604574,78.953192,8.394770,4.224211,4.330997,7.404501,0.500132,10.901690,0.448935,...,2.079169,9.461168,2.173070,2.538807,3.497500,6.927808,10.268626,2.439296,1.356477,5.120317
22284,209.767417,2.619164,78.722606,0.502380,6.121796,2.877773,8.158987,1.029359,4.642778,0.180140,...,1.254242,4.993181,3.761897,7.348515,2.770520,6.943558,5.523586,0.602111,3.575212,7.056725


In [36]:
pc_sortino_bottom10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).tail(10)

In [37]:
pc_sortino_bottom10.to_csv('Sortino_Bottom10_MV.csv')

In [38]:
pc_sortino_bottom10

,Returns,Semi-Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
44883,76.565901,7.492181,9.741610,9.643415,1.820759,1.344446,4.997183,6.740643,2.829254,1.650644,...,5.147259,1.581401,2.238978,2.942618,6.574603,3.390945,0.537417,6.684559,5.034409,2.279622
26654,80.596201,8.024496,9.597637,4.795562,1.372689,6.383152,2.160679,9.358414,1.370630,2.686597,...,8.024273,2.291254,10.500394,3.556650,1.864019,2.790852,0.201059,9.056558,2.196084,3.785736
20431,77.897853,7.792962,9.536534,6.881045,1.910342,1.076953,5.654663,8.201898,2.334778,4.584685,...,4.375085,1.148222,4.610567,0.193633,4.672578,0.791802,3.406732,9.067687,3.257843,1.162243
9598,81.496213,8.170619,9.536145,5.828475,7.368185,4.809575,0.494485,2.023215,1.247807,6.080714,...,10.293061,2.281793,1.150556,3.164356,3.882174,0.569241,1.663514,10.461246,6.094095,1.864769
2830,76.787363,7.771629,9.419822,3.653483,9.942480,0.385554,0.507765,7.761938,4.534854,0.897708,...,8.463120,10.554607,2.743852,3.974217,10.558167,1.654889,0.442538,6.202314,3.555804,0.386639
38467,66.316229,6.677083,9.395754,8.580405,7.185264,4.165097,7.375508,4.106352,0.960109,0.171980,...,7.641789,1.360662,0.380875,1.433416,8.186388,5.199518,1.097556,8.231694,3.589004,0.265221
45822,77.532147,7.981597,9.265332,9.029674,0.823211,1.382259,3.639516,3.402557,1.963243,6.870039,...,9.365914,3.303262,1.363581,8.231841,8.570805,0.002298,2.607312,7.640750,1.095483,1.837660
18389,83.369785,8.638440,9.236596,5.497131,0.772793,1.058609,4.187195,6.916688,3.241200,6.016361,...,8.474405,1.114450,5.424558,0.019835,4.835515,2.778165,0.714162,6.040497,5.716636,4.449858
31313,71.500542,7.876817,8.622842,7.742250,1.014548,0.072420,6.478392,3.924452,7.952237,2.366683,...,5.925598,0.827911,7.587622,2.341137,7.391614,0.455043,3.357966,10.405721,7.445282,1.060810
25277,58.630572,7.570031,7.272173,3.604254,5.889530,0.731506,6.177342,3.333757,9.905012,6.438556,...,8.576446,1.046964,1.202763,0.277483,3.836444,1.263206,0.722488,10.108781,2.299703,4.022140


In [39]:
sortino_optimal_portfolio=pc_sortino_top10.head(1)

In [40]:
sortino_optimal_portfolio.to_csv('Sortino_Optimal_MV.csv')

In [41]:
sortino_optimal_portfolio.T

,7455
Returns,226.315055
Semi-Deviation,2.478782
Sortino Ratio,89.856660
ASIANPAINT Weight,9.015889
BAJFINANCE Weight,0.544998
BAJAJFINSV Weight,9.081724
BRITANNIA Weight,7.232069
DIVISLAB Weight,0.023027
GRASIM Weight,8.179124
HCLTECH Weight,3.668750


In [ ]:
edf=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
edf = edf.loc["2016-01-01" : ]   
edf.reset_index(drop=True, inplace=True)
(edf / edf.iloc[0]*100).plot(figsize=(10,5))


In [ ]:
plt.style.use('seaborn')
pc_sharpe.plot.scatter(x='Standard Deviation', y='Returns', marker='o',figsize=(20, 10), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
#GREEN STAR-> Optimal Risky Portfolio
#RED STAR-> Minimum Volatility
plt.subplots(figsize=(20, 10))
plt.xlabel("Expected Volatility")
plt.ylabel("Returns")
plt.scatter(pc_sharpe['Standard Deviation'], pc_sharpe['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(minimum_risk_portfolio['Standard Deviation'], minimum_risk_portfolio['Returns'], color='r', marker='*', s=500)
plt.scatter(sharpe_optimal_portfolio['Standard Deviation'], sharpe_optimal_portfolio['Returns'], color='g', marker='*', s=500)
